In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=d7ce32446f8beb0370044a8715be36864470bb63e776285a02641ad2924380f5
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0977 ,random_state=0 , l1_ratio=0.2605, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 10.83 , C = 0.0001 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_2000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_16_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=929009, Mon Apr 13 14:48:04 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 -1.794752858843314e+02 1.0e+00 2.58e-01  3e-01  3e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 14:48:04 2020)
final/bestever f-value = -1.794753e+02 -1.794753e+02
incumbent solution: [0.16180244 0.79996601 0.10362123 0.79736258 0.60837682 0.87307026
 1.22314271 0.3953869  ...]
std deviations: [0.25747957 0.2583059  0.25738707 0.25779122 0.2573467  0.2576886
 0.25815607 0.2574114  ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=857779, Mon Apr 13 14:48:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 -1.798314889749156e+02 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 14:48:05 2020)
final/bestever f-value = -1.798315e+02 -1.798315e+02
incumbent solution: [-0.07733162  0.92810179  0.09877303  0.7027788   0.612

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.83148897] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.831489, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more e

    3     45 -1.812999853688674e+02 1.0e+00 2.39e-01  2e-01  2e-01 0:00.1
   99   1485 -9.909608977247376e+06 1.4e+00 2.75e+02  3e+02  3e+02 0:01.9
termination on tolfacupx=1000.0 (Mon Apr 13 14:48:07 2020)
final/bestever f-value = -9.123951e+06 -9.909609e+06
incumbent solution: [-397.55476081   14.47389425  282.99634467  -65.49232562 2470.1169658
 -182.80631264 -475.14409924 -795.07677803 ...]
std deviations: [267.77226563 263.58203538 268.4113243  262.55862249 284.36148292
 264.05724034 268.60604554 269.18356588 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=917647, Mon Apr 13 14:48:07 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 -1.794752858843504e+02 1.0e+00 2.52e-01  3e-01  3e-01 0:00.0
    2     30 -1.794752858845024e+02 1.0e+00 2.40e-01  2e-01  2e-01 0:00.0
    3     45 -1.794752859013670e+02 1.1e+00 2.33e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.908610938647322e+02 1.4e+00 2.14e-02  2e-02  2e-02 0:02.0
  200   3000 -1.913595296007734e+02 1.4e+00 1.25e-03  1e-03  1e-03 0:03.9
  300   4500 -1.913629826637133e+02 1.5e+00 7.18e-05  6e-05  7e-05 0:05.8
  400   6000 -1.913629918348963e+02 1.5e+00 4.09e-06  3e-06  4e-06 0:07.7
  500   7500 -1.913629918573378e+02 1.5e+00 2.42e-07  2e-07  2e-07 0:09.7
  600   9000 -1.913629918573975e+02 1.5e+00 1.31e-08  1e-08  1e-08 0:11.6
  643   9645 -1.913629918573976e+02 1.6e+00 4.52e-09  3e-09  4e-09 0:12.4
termination on tolfun=1e-11 (Mon Apr 13 14:48:21 2020)
final/bestever f-value = -1.913630e+02 -1.913630e+02
incumbent solution: [ 0.42285705  0.11974751  0.17525152  0.29317548 -0.01866462  0.02025541
  0.01668744  0.00488567 ...]
std deviations: [3.66747949e-09 3.56019188e-09 3.50978286e-09 3.56739388e-09
 3.47202605e-09 3.75673473e-09 3.52541733e-09 3.56115537e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=951353, Mon Apr 13 14:48:21 2020)
Iterat #Fevals  

...

    3     45 -1.808856167655165e+02 1.1e+00 2.43e-01  2e-01  2e-01 0:00.1
  100   1500 -4.574067821375979e+06 1.5e+00 1.67e+02  2e+02  2e+02 0:01.9
  106   1590 -1.272661710319634e+07 1.5e+00 2.74e+02  3e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 14:52:43 2020)
final/bestever f-value = -1.210982e+07 -1.272662e+07
incumbent solution: [ 480.80903012  506.17926918  390.64216363 -213.73713749 3191.45320592
   15.95356603  378.64063243  340.4508244  ...]
std deviations: [269.79511473 261.3607781  267.02174719 262.45161832 279.52866275
 265.99451105 262.79914215 268.17287465 ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=896100, Mon Apr 13 14:52:43 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 -1.794752858843480e+02 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
    2     30 -1.794752858843364e+02 1.0e+00 2.46e-01  2e-01  2e-01 0:00.0
    3     45 -1.794752858849224e+02 1.1e+00 2.38e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -1.888731657631246e+02 1.4e+00 2.09e-02  2e-02  2e-02 0:02.0
  200   3000 -1.894758571493129e+02 1.4e+00 1.07e-03  9e-04  1e-03 0:04.0
  300   4500 -1.894770773947278e+02 1.5e+00 5.20e-05  4e-05  5e-05 0:06.0
  400   6000 -1.894770806774311e+02 1.5e+00 3.13e-06  3e-06  3e-06 0:07.9
  500   7500 -1.894770806941954e+02 1.6e+00 1.78e-07  1e-07  2e-07 0:09.9
  600   9000 -1.894770806942391e+02 1.6e+00 1.20e-08  9e-09  1e-08 0:11.9
  637   9555 -1.894770806942392e+02 1.6e+00 5.06e-09  4e-09  4e-09 0:12.6
termination on tolfun=1e-11 (Mon Apr 13 14:52:58 2020)
final/bestever f-value = -1.894771e+02 -1.894771e+02
incumbent solution: [-0.29571981  0.46396192  0.43892821  0.21340087  0.38812173  0.14205786
  0.28665624  0.03409737 ...]
std deviations: [4.04292153e-09 3.86286404e-09 3.98482062e-09 4.00458033e-09
 3.83549672e-09 4.42477619e-09 3.93819827e-09 4.01549162e-09 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=873497, Mon Apr 13 14:52:58 2020)
Iterat #Fevals  

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.83148897] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.831489, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.831489, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -6.304377878770076e+06 1.3e+00 2.48e+02  2e+02  2e+02 0:01.9
  102   1530 -7.385351345102656e+06 1.3e+00 2.83e+02  3e+02  3e+02 0:01.9
termination on tolfacupx=1000.0 (Mon Apr 13 14:53:00 2020)
final/bestever f-value = -6.906686e+06 -7.385351e+06
incumbent solution: [-347.0857107   523.76927252  513.32396164  474.41555393 2226.54541133
 1177.39762733  206.26842524 -295.08705286 ...]
std deviations: [273.25986256 274.73335917 274.4849537  275.35113618 273.17029748
 280.40656406 274.51827286 275.58849096 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F16_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:50])
    SVM_Fun [i] = F16(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F16(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-197.8344297]), count=array([1])) 13.15946889141258
SVM
ModeResult(mode=array([-206.62575231]), count=array([1])) 14.073514321699802
ELN
ModeResult(mode=array([4423649.14002159]), count=array([1])) 689951.7641337649
